# **Data Collection**
---

## Objectives

* Answer Business Requirement 1:
  * The client is interested in conducting a study to visually differentiate a healthy cherry leaf from one with powdery mildew.

## Inputs

* inputs/cherry_leaves_dataset/cherry-leaves/train
* inputs/cherry_leaves_dataset/cherry-leaves/validation
* inputs/cherry_leaves_dataset/cherry-leaves/test

## Outputs

* Image shape embeddings pickle file
* Mean and variability of images per label plot
* Plot to distinguish contrast between healthy and powdery mildew-infected cherry leaves.
* Generate code to answer business requirement 1 and can be used to build a gallery of images on the Streamlit dashboard

## Additional Comments

* No comments
---